<a href="https://colab.research.google.com/github/callumselv/Y4_project/blob/main/Lohani_2021.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install qutip

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.0/28.0 MB 20.2 MB/s eta 0:00:00


In [3]:
import numpy as np
from qutip import *
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
import matplotlib.pyplot as plt
from scipy.stats import unitary_group

In [5]:
no_data = 10000
d = 2
dimension = 2**d

In [6]:
# Generate rho matrices

rhos = np.zeros((no_data,dimension,dimension),dtype=complex)
eta = 1e-7

for i in range(no_data):
    random_unitary = unitary_group.rvs(dimension)
    coeffs = random_unitary[:,0]
    rho = np.outer(coeffs,coeffs.conj())
    rhos[i] = (1-eta)*rho + (eta/4)*np.eye(dimension)

In [7]:
cholesky_decompositions = np.array([np.linalg.cholesky(rho) for rho in rhos])

In [17]:
cholesky_decompositions[1000]

array([[ 4.91545828e-01+0.00000000e+00j,  0.00000000e+00+0.00000000e+00j,
         0.00000000e+00+0.00000000e+00j,  0.00000000e+00+0.00000000e+00j],
       [ 4.00641600e-01+2.05830417e-01j,  2.14457131e-04+0.00000000e+00j,
         0.00000000e+00+0.00000000e+00j,  0.00000000e+00+0.00000000e+00j],
       [-5.92334409e-01-3.66851128e-01j, -1.50928101e-04-1.20885052e-05j,
         2.28697950e-04+0.00000000e+00j,  0.00000000e+00+0.00000000e+00j],
       [ 1.61215538e-01+2.09934376e-01j,  5.20106684e-05+2.45700384e-05j,
        -4.24246679e-05-1.60368105e-05j,  1.63962004e-04+0.00000000e+00j]])

In [ ]:
all_taus = np.zeros((no_data, dimension**2))
for no in range(no_data):
    for cholesky in cholesky_decompositions:
        extracted_values = []
        for i in range(cholesky.shape[0]):
            extracted_values.append(cholesky[i, i].real)

        primary_indices = [(1, 0), (2, 1), (3, 2)]
        for i, j in primary_indices:
            extracted_values.append(cholesky[i, j].real)
            extracted_values.append(cholesky[i, j].imag)

        secondary_indices = [(2, 0), (3, 1), (3, 0)]
        for i, j in secondary_indices:
            extracted_values.append(cholesky[i, j].real)
            extracted_values.append(cholesky[i, j].imag)
    all_taus[no] = np.array(extracted_values)


In [18]:
all_taus.shape

array([ 5.77218028e-01,  1.66564747e-04,  1.61222272e-04,  2.55012802e-04,
       -1.89842567e-01, -2.29616664e-02,  3.66482186e-06,  9.77301446e-06,
       -1.97556162e-05,  3.46289058e-05, -2.86126119e-02, -1.17737447e-01,
       -6.67272630e-05, -1.07462703e-05,  7.54018743e-01,  2.16856479e-01])

In [38]:
# Preallocate array for all extracted values, assuming each extraction yields the same size
tau_size = dimension**2  # Assuming this based on `all_taus` shape
all_taus = np.zeros((no_data, tau_size))

primary_indices = [(1, 0), (2, 1), (3, 2)]
secondary_indices = [(2, 0), (3, 1), (3, 0)]

for no in range(no_data):
    cholesky = cholesky_decompositions[no]
    extracted_values = np.zeros(tau_size)

    for i in range(cholesky.shape[0]):
        extracted_values[i] = cholesky[i, i].real

    index = cholesky.shape[0]
    for i, j in primary_indices:
        extracted_values[index] = cholesky[i, j].real
        extracted_values[index + 1] = cholesky[i, j].imag
        index += 2

    for i, j in secondary_indices:
        extracted_values[index] = cholesky[i, j].real
        extracted_values[index + 1] = cholesky[i, j].imag
        index += 2

    all_taus[no] = extracted_values


In [36]:
all_taus.shape[1]

16

In [29]:
def reconstruct_matrix(taus):
    matrix = np.zeros((4, 4), dtype=np.complex_)

    index = 0
    for i in range(4):
        matrix[i, i] = taus[index]
        index += 1

    primary_indices = [(1, 0), (2, 1), (3, 2)]
    for i, j in primary_indices:
        real_part = taus[index]
        imag_part = taus[index + 1]
        matrix[i, j] = real_part + 1j * imag_part
        index += 2

    secondary_indices = [(2, 0), (3, 1), (3, 0)]
    for i, j in secondary_indices:
        real_part = taus[index]
        imag_part = taus[index + 1]
        matrix[i, j] = real_part + 1j * imag_part
        index += 2

    return matrix

In [30]:
# Make 6^d projectors of Pauli {X,Y,Z} operators

paulis = [sigmax(),sigmay(),sigmaz()]
evectors = np.zeros((6,2,1),dtype=complex)

i=0
for pauli in paulis:
    vals, vecs = pauli.eigenstates()
    evectors[i] = vecs[0][:]
    evectors[i+1] = vecs[1][:]
    i += 2

In [31]:
projectors = np.zeros((6,2,2), dtype=complex)

for i, vectors in enumerate(evectors):
    projectors[i] = np.outer(vectors,vectors.conj())

In [32]:
pauli_2d = np.zeros((6**2,4,4), dtype=complex)

index=0
for i in projectors:
    for j in projectors:
        pauli_2d[index] = np.kron(i,j)
        index += 1

In [42]:
pauli_2d.shape

(36, 4, 4)

In [46]:
((rho[0])@pauli_2d).shape

(36, 4)

In [47]:
nvals = np.zeros((no_data,6**2))

for i1,rho in enumerate(rhos):
  for index,proj in enumerate(pauli_2d):
      nvals[i1,index] = np.trace(rho@proj)

<ipython-input-47-09091a42e478>:5: ComplexWarning: Casting complex values to real discards the imaginary part
  nvals[i1,index] = np.trace(rho@proj)


In [53]:
def build_cnn_model():
    model = Sequential()

    model.add(tf.keras.layers.Reshape((6, 6, 1), input_shape=(36,)))

    model.add(Conv2D(filters=25, kernel_size=(2, 2), strides=1, activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(filters=25, kernel_size=(2, 2), strides=1, activation='relu'))

    model.add(Flatten())

    model.add(Dense(750, activation='relu'))
    model.add(Dropout(0.5))

    model.add(Dense(450, activation='relu'))
    model.add(Dropout(0.5))

    model.add(Dense(tau_size, activation='linear'))

    return model

model = build_cnn_model()

model.compile(
    optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.005),
    loss='mean_squared_error',
    metrics=['mean_absolute_error']
)

model.summary()


/usr/local/lib/python3.10/dist-packages/keras/src/layers/reshaping/reshape.py:39: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ reshape_3 (Reshape)                  │ (None, 6, 6, 1)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_6 (Conv2D)                    │ (None, 5, 5, 25)            │             125 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_6 (MaxPooling2D)       │ (None, 2, 2, 25)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_7 (Conv2D)                    │ (None, 1, 1, 25)            │           2,525 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_3 (Flatten)                  │ (None, 25)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_8 (Dense)                      │ (None, 750)                 │          19,500 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_6 (Dropout)                  │ (None, 750)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_9 (Dense)                      │ (None, 450)                 │         337,950 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_7 (Dropout)                  │ (None, 450)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_10 (Dense)                     │ (None, 16)                  │           7,216 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 367,316 (1.40 MB)

 Trainable params: 367,316 (1.40 MB)

 Non-trainable params: 0 (0.00 B)

In [54]:
# `x_train`, `y_train` are probabilities (n values) and labels (tau-vectors)
model.fit(nvals, all_taus, epochs=300, batch_size=100, validation_split=0.2)

Epoch 1/300
80/80 ━━━━━━━━━━━━━━━━━━━━ 5s 7ms/step - loss: 0.0615 - mean_absolute_error: 0.1471 - val_loss: 0.0568 - val_mean_absolute_error: 0.1346
Epoch 2/300
80/80 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.0568 - mean_absolute_error: 0.1409 - val_loss: 0.0540 - val_mean_absolute_error: 0.1302
Epoch 3/300
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0542 - mean_absolute_error: 0.1371 - val_loss: 0.0524 - val_mean_absolute_error: 0.1277
Epoch 4/300
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0529 - mean_absolute_error: 0.1360 - val_loss: 0.0514 - val_mean_absolute_error: 0.1261
Epoch 5/300
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0520 - mean_absolute_error: 0.1348 - val_loss: 0.0508 - val_mean_absolute_error: 0.1249
Epoch 6/300
80/80 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.0513 - mean_absolute_error: 0.1340 - val_loss: 0.0504 - val_mean_absolute_error: 0.1240
Epoch 7/300
80/80 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.0510 - mean_absolute_error: 0.1337 - val_loss